<a href="https://colab.research.google.com/github/neohack22/ML-engineering/blob/master/01-entity-extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entity Extraction with Claude

> *This notebook should work well with the **`Python 3`** kernel in SageMaker Studio*

### Choosing an approach

![image.png](attachment:image.png)

### Context
Entity extraction is an NLP technique that allows us to automatically extract specific data from naturally written text, such as news, emails, books, etc.
That data can then later be saved to a database, used for lookup or any other type of processing.

Classic entity extraction programs usually limit you to pre-defined classes, such as name, address, price, etc. or require you to provide many examples of types of entities you are interested in.
By using a LLM for entity extraction in most cases you are only required to specify what you need to extract in natural language. This gives you flexibility and accuracy in your queries while saving time by removing necessity of data labeling.

In addition, LLM entity extraction can be used to help you assemble a dataset to later create a customised solution for your use case, such as [Amazon Comprehend custom entity](https://docs.aws.amazon.com/comprehend/latest/dg/custom-entity-recognition.html) recognition.

## Setup


In [ ]:
# %pip install -U langchain-aws=='0.1.17'

In [ ]:
import json
import os
import sys

import boto3
import botocore

boto3_bedrock = boto3.client('bedrock-runtime')

Ce code importe plusieurs modules Python et initialise un client Boto3 pour interagir avec Amazon Bedrock. Voici une explication détaillée :

1. Les importations :
   - `json` : pour travailler avec des données JSON
   - `os` et `sys` : pour interagir avec le système d'exploitation
   - `boto3` et `botocore` : pour interagir avec les services AWS

2. Initialisation du client Bedrock :
   ```python
   boto3_bedrock = boto3.client('bedrock-runtime')
   ```
   Cette ligne crée un client Boto3 spécifiquement pour le service Amazon Bedrock Runtime.

Ce code est le début d'un script qui va interagir avec Amazon Bedrock, probablement pour utiliser des modèles d'IA ou effectuer des tâches liées à l'apprentissage automatique.

Il est important de noter que pour que ce code fonctionne correctement, vous devez avoir configuré vos identifiants AWS correctement, soit via des variables d'environnement, soit via un fichier de configuration AWS.

## Configure langchain

We begin with instantiating the LLM. Here we are using Anthropic Claude v3 for text generation.

Note: It is possible to choose other models available with Bedrock. For example, you can replace the `model_id` as follows to change the model to Titan Text Premier. Make sure your account has access to the model you want to try out before trying this!

`llm = ChatBedrock(model_id="amazon.titan-text-premier-v1:0")`

Check [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) for Available text generation model Ids under Amazon Bedrock.

In [ ]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs={
        "max_tokens": 200,
        "temperature": 0, # Using 0 to get reproducible results
        "stop_sequences": ["\n\nHuman:"]
    }
)

La question implicite dans ce code concerne l'utilisation de la bibliothèque `langchain_aws` et plus particulièrement la classe `ChatBedrock` pour interagir avec le modèle Claude 3 d'Anthropic via AWS Bedrock.

Voici une explication du code :

```markdown
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs={
        "max_tokens": 200,
        "temperature": 0,
        "stop_sequences": ["\n\nHuman:"]
    }
)
```

Ce code initialise un objet `llm` de type `ChatBedrock` qui permet d'interagir avec le modèle de langage Claude 3 d'Anthropic via AWS Bedrock. Voici les détails des paramètres utilisés :

1. `model_id` : Spécifie la version exacte du modèle Claude 3 à utiliser.
2. `model_kwargs` : Un dictionnaire contenant des paramètres supplémentaires pour le modèle :
   - `max_tokens` : Limite la réponse à 200 tokens maximum.
   - `temperature` : Réglée à 0 pour obtenir des résultats reproductibles (moins de créativité/variabilité).
   - `stop_sequences` : Indique au modèle d'arrêter la génération lorsqu'il rencontre la séquence "\n\nHuman:".

Ce code configure essentiellement un client pour interagir avec le modèle Claude 3 via AWS Bedrock, en définissant des paramètres spécifiques pour contrôler la génération de texte.

## Entity Extraction
Now that we have our LLM initialised, we can start extracting entities.

For this exercise we will pretend to be an online bookstore that receives questions and orders by email.
Our task would be to extract relevant information from the email to process the order.

Let's begin by taking a look at the sample email:

In [ ]:
from pathlib import Path

emails_dir = Path(".") / "emails"
with open(emails_dir / "00_treasure_island.txt") as f:
    book_question_email = f.read()

print(book_question_email)

1. Il importe le module `Path` de la bibliothèque `pathlib`.
2. Il crée un chemin vers un répertoire nommé "emails" dans le répertoire courant.
3. Il ouvre un fichier spécifique nommé "00_treasure_island.txt" dans ce répertoire.
4. Il lit le contenu du fichier.
5. Enfin, il imprime le contenu du fichier.

Voici une explication plus détaillée :

```markdown
from pathlib import Path
```
Cette ligne importe la classe `Path` du module `pathlib`, qui fournit une interface orientée objet pour travailler avec les chemins de fichiers.

```markdown
emails_dir = Path(".") / "emails"
```
Cette ligne crée un objet `Path` représentant le répertoire "emails" dans le répertoire courant (représenté par ".").

```markdown
with open(emails_dir / "00_treasure_island.txt") as f:
    book_question_email = f.read()
```
Ce bloc ouvre le fichier "00_treasure_island.txt" dans le répertoire "emails", lit son contenu et le stocke dans la variable `book_question_email`.

```markdown
print(book_question_email)
```
Cette dernière ligne imprime le contenu du fichier.

Ce code est utilisé pour lire le contenu d'un fichier spécifique dans un répertoire donné et l'afficher. Il utilise la bibliothèque `pathlib` pour gérer les chemins de fichiers de manière plus robuste et portable entre différents systèmes d'exploitation.

### Basic approach

For basic cases we can directly ask the model to return the result.
Let's try extracting the name of the book.

In [ ]:
query = f"""
Given the email inside triple-backticks, please read it and analyse the contents.
If a name of a book is mentioned, return it, otherwise return nothing.

Email: ```
{book_question_email}
```

"""

messages = [
    (
        "system",
        "You are a helpful assistant that processes orders from a bookstore.",
    ),
    ("human", query),
]

La question Python dans ce texte demande de lire et analyser le contenu d'un email contenu dans des backticks (```...```), et de vérifier si un nom de livre y est mentionné. Si un nom de livre est trouvé, il doit être retourné. Sinon, la fonction doit retourner une chaîne vide ou rien.

Voici une solution possible en Python pour répondre à cette question :

```python
import re

def find_book_in_email(email_content):
    # Utilisation d'une expression régulière pour identifier un possible nom de livre
    # On suppose qu'un nom de livre pourrait être entre guillemets ou être en majuscules.
    match = re.search(r'\"(.+?)\"|(?:[A-Z][a-z]+\s){1,3}[A-Z][a-z]+', email_content)
    
    if match:
        return match.group(0)
    return None

# Exemple d'email
book_question_email = """
Je vous contacte au sujet de votre livre intitulé "Les Secrets du Python". J'ai des questions sur le chapitre 3.
"""

# Appel de la fonction
result = find_book_in_email(book_question_email)

print(result)  # Affichera le nom du livre s'il est trouvé
```

### Explication :
- L'expression régulière utilisée dans `re.search` cherche un texte entre guillemets ou une séquence de mots avec une majuscule suivie de mots avec des minuscules (qui pourrait représenter un nom de livre).
- Si un match est trouvé, il est retourné, sinon la fonction retourne `None`.

In [ ]:
result = llm.invoke(messages)
print(result.content)

Dans ce texte, la question Python concerne l'appel d'un modèle de langage (`llm`) pour traiter des messages et renvoyer le contenu de la réponse.

Le code suivant montre comment un modèle d'IA pourrait être invoqué, et comment son résultat (contenu) peut être imprimé en Python. Il n'y a pas de contexte direct sur la structure de `llm`, mais cela ressemble à un modèle d'IA comme GPT utilisé via une interface API.

Voici une réponse avec un exemple de code Python qui reflète la logique décrite :

```python
# Simulation d'appel à un modèle de langage pour traiter des messages
class LLM:
    def invoke(self, messages):
        # Simuler une réponse basée sur le contenu du message
        return self.Response(f"Réponse du modèle pour le message: {messages[-1][1]}")
    
    class Response:
        def __init__(self, content):
            self.content = content

# Exemple d'utilisation
llm = LLM()
messages = [
    ("system", "Vous êtes un assistant utile."),
    ("human", "Quel est le nom de ce livre ?"),
]

# Appel du modèle et impression de la réponse
result = llm.invoke(messages)
print(result.content)
```

### Explication :
- La classe `LLM` simule un modèle de langage avec une méthode `invoke` qui prend une liste de messages et retourne une réponse simulée.
- Le résultat de `llm.invoke(messages)` est un objet `Response`, dont l'attribut `content` contient la réponse.
- Le contenu de la réponse est ensuite imprimé avec `print(result.content)`.

Le véritable comportement dépendra de l'implémentation du modèle de langage et de l'API utilisée, mais cette structure de base illustre bien le mécanisme d'invocation et de récupération du résultat.

### Model specific prompts

While basic approach works, to achieve best results we recommend to customise your prompts for the particular model you will be using.
In this example we are using `anthropic.claude-3`, [prompt guide for which can be found here](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design).

Here is the a more optimised prompt for Claude v3.

In [ ]:
prompt = """

Given the email provided, please read it and analyse the contents.
If a name of a book is mentioned, return it.
If no name is mentioned, return empty string.
The email will be given between <email></email> XML tags.

<email>
{email}
</email>

Return the name of the book between <book></book> XML tags.

"""

Dans ce texte, la question Python consiste à lire un email encadré par des balises XML `<email></email>`, puis à analyser son contenu pour voir si un nom de livre y est mentionné. Si un livre est mentionné, le nom doit être renvoyé entre balises `<book></book>`. Sinon, une chaîne vide doit être retournée.

Voici une solution en Python pour accomplir cette tâche :

```python
import re

def find_book_in_email(email_content):
    # Extraction du contenu entre les balises <email></email>
    email_match = re.search(r'<email>(.*?)</email>', email_content, re.DOTALL)
    
    if email_match:
        email_text = email_match.group(1)
        
        # Recherche d'un nom de livre potentiel entre guillemets
        book_match = re.search(r'\"(.+?)\"', email_text)
        
        if book_match:
            # Retourne le nom du livre entre balises <book></book>
            return f"<book>{book_match.group(1)}</book>"
    
    # Si aucun livre n'est trouvé, retourne une chaîne vide
    return "<book></book>"

# Exemple d'email
email = """
<email>
Je voudrais vous poser une question sur le livre "Apprendre Python en 30 jours". Est-il encore disponible ?
</email>
"""

# Appel de la fonction
result = find_book_in_email(email)

print(result)  # Affichera le nom du livre s'il est trouvé, ou une chaîne vide sinon
```

### Explication :
- On utilise `re.search` pour extraire le contenu entre les balises `<email></email>`.
- Une deuxième recherche (`re.search`) est utilisée pour trouver un texte entre guillemets, qui pourrait représenter un nom de livre.
- Si un livre est trouvé, il est encapsulé entre les balises `<book></book>`.
- Si aucun livre n'est trouvé, une balise vide `<book></book>` est retournée.

In [ ]:
query = prompt.format(email=book_question_email)
messages = [
    (
        "system",
        "You are a helpful assistant that processes orders from a bookstore.",
    ),
    ("human", query),
]
result = llm.invoke(messages).content
print(result)

La question Python ici concerne l'envoi d'une requête formatée à un modèle de langage (`llm`) pour analyser le contenu d'un email et renvoyer un résultat. L'email est formaté dans une variable `query`, puis inséré dans une structure de message envoyée au modèle.

Voici une solution qui montre comment cela pourrait fonctionner dans un scénario Python typique, en utilisant un modèle de langage fictif `llm` pour simuler l'appel :

```python
class LLM:
    def invoke(self, messages):
        # Simulation de la réponse du modèle
        return self.Response(f"Le livre trouvé est : 'Python pour les débutants'.")
    
    class Response:
        def __init__(self, content):
            self.content = content

# Initialisation du modèle de langage
llm = LLM()

# Formatage de la requête avec l'email
book_question_email = """
Je suis intéressé par le livre 'Python pour les débutants'. Pouvez-vous m'en dire plus ?
"""
prompt = """
Given the email provided, please read it and analyse the contents.
If a name of a book is mentioned, return it.
If no name is mentioned, return empty string.
The email will be given between <email></email> XML tags.

<email>
{email}
</email>

Return the name of the book between <book></book> XML tags.
"""

# Formatage de la requête
query = prompt.format(email=book_question_email)

# Création des messages à envoyer au modèle
messages = [
    ("system", "You are a helpful assistant that processes orders from a bookstore."),
    ("human", query),
]

# Invocation du modèle et récupération du résultat
result = llm.invoke(messages).content

# Affichage du résultat
print(result)
```

### Explication :
- `LLM` est une classe simulant le modèle de langage. Elle contient une méthode `invoke` qui reçoit des messages et renvoie une réponse simulée.
- Le `prompt` contient la logique pour formater un email dans une structure XML, où le contenu de l'email est inséré à l'aide de `format()`.
- `query` est le texte final envoyé au modèle après le formatage avec le contenu réel de l'email.
- `llm.invoke(messages)` appelle le modèle avec les messages et renvoie un objet `Response` contenant la réponse, qui est ensuite imprimée.

Dans un vrai environnement, le modèle de langage renverrait une analyse basée sur le contenu de l'email.

To extract results easier, we can use a helper function:

In [ ]:
from bs4 import BeautifulSoup

def extract_by_tag(response: str, tag: str, extract_all=False) -> str | list[str] | None:
    soup = BeautifulSoup(response)
    results = soup.find_all(tag)
    if not results:
        return

    texts = [res.get_text() for res in results]
    if extract_all:
        return texts
    return texts[-1]

La question Python ici porte sur l'extraction de texte à partir de balises spécifiques dans une réponse HTML à l'aide de BeautifulSoup. Le code fourni définit une fonction `extract_by_tag` qui accepte trois paramètres : la réponse HTML sous forme de chaîne (`response`), le nom de la balise (`tag`) à extraire, et un booléen (`extract_all`) qui détermine s'il faut extraire tous les éléments correspondants ou seulement le dernier.

Voici une explication de ce que fait ce code et comment l'améliorer :

1. **Analyse de la structure :**
   - `BeautifulSoup(response)` : Analyse la réponse HTML pour permettre une manipulation facile des balises.
   - `soup.find_all(tag)` : Trouve toutes les occurrences de la balise spécifiée dans le document.
   - Si `extract_all` est vrai, la fonction retourne une liste de textes correspondant à chaque balise trouvée.
   - Sinon, elle retourne uniquement le texte de la dernière balise trouvée.

2. **Amélioration** : Pour plus de robustesse, il pourrait être utile de spécifier le parser utilisé par BeautifulSoup (comme `html.parser`) et de traiter correctement les valeurs `None`.

Voici une version améliorée du code :

```python
from bs4 import BeautifulSoup
from typing import Union, List, Optional

def extract_by_tag(response: str, tag: str, extract_all: bool = False) -> Union[str, List[str], None]:
    # Analyse du contenu HTML avec le parser HTML standard
    soup = BeautifulSoup(response, 'html.parser')
    
    # Recherche de toutes les balises correspondant au tag spécifié
    results = soup.find_all(tag)
    
    if not results:
        return None  # Retourne None si aucune balise n'est trouvée
        
    # Extrait le texte de chaque balise trouvée
    texts = [res.get_text() for res in results]
    
    # Retourne tous les textes si extract_all est True, sinon retourne seulement le dernier texte
    if extract_all:
        return texts
    return texts[-1] if texts else None

# Exemple d'utilisation
response_html = """
<html>
  <body>
    <p>Premier paragraphe</p>
    <p>Deuxième paragraphe</p>
    <p>Dernier paragraphe</p>
  </body>
</html>
"""

# Extraction du dernier paragraphe
print(extract_by_tag(response_html, 'p'))

# Extraction de tous les paragraphes
print(extract_by_tag(response_html, 'p', extract_all=True))
```

### Explication des améliorations :
- **Parser HTML explicite** : `'html.parser'` est spécifié pour que BeautifulSoup sache quel parser utiliser.
- **Utilisation de `Optional` et `Union` dans les annotations de type** pour mieux documenter le fait que la fonction peut renvoyer une chaîne, une liste de chaînes ou `None`.
- **Robustesse** : La fonction retourne `None` si aucune balise n'est trouvée, pour gérer les cas où le tag est absent du document.

In [ ]:
extract_by_tag(result, "book")

La question Python ici concerne l'utilisation de la fonction `extract_by_tag` pour extraire du texte d'une balise `<book>` dans la variable `result`. La fonction a probablement été définie pour prendre un contenu HTML (ou similaire) et en extraire le texte se trouvant à l'intérieur d'une balise donnée, ici `"book"`.

Si nous utilisons la fonction définie précédemment, voici comment vous pourriez l'appliquer à l'extraction du contenu à l'intérieur de la balise `<book>` de la variable `result` :

### Exemple d'utilisation :

```python
# Supposons que 'result' contient le HTML ou XML retourné
result = """
<response>
    <book>Apprendre Python</book>
    <book>Python Avancé</book>
</response>
"""

# Extraction de la dernière balise <book>
book_name = extract_by_tag(result, "book")

# Affichage du résultat
print(book_name)  # Affichera 'Python Avancé'
```

### Explication :
- `extract_by_tag(result, "book")` appelle la fonction pour chercher toutes les balises `<book>` dans la chaîne `result`.
- Si plusieurs balises `<book>` existent, la fonction (avec `extract_all=False` par défaut) renvoie uniquement le texte de la dernière balise trouvée.
- Dans cet exemple, cela retournerait "Python Avancé" car c'est le dernier livre dans la balise `<book>`.

Si vous vouliez extraire **tous** les livres, vous pouvez passer l'argument `extract_all=True` :

```python
# Extraction de toutes les balises <book>
all_books = extract_by_tag(result, "book", extract_all=True)

# Affichage de tous les résultats
print(all_books)  # Affichera ['Apprendre Python', 'Python Avancé']
```

We can check that our model doesn't return arbitrary results when no appropriate information is given (also know as 'hallucination'), by running our prompt on other emails.

In [ ]:
with open(emails_dir / "01_return.txt") as f:
    return_email = f.read()

print(return_email)

La question Python ici consiste à lire le contenu d'un fichier texte nommé `"01_return.txt"` situé dans un répertoire `emails_dir` et à afficher ce contenu. Le fichier est ouvert avec `open()`, lu entièrement avec `f.read()`, puis imprimé avec `print()`.

Voici une explication et un exemple complet de cette opération en Python :

### Explication :
1. **Chemin du fichier** : `emails_dir / "01_return.txt"` indique que le fichier se trouve dans le répertoire `emails_dir`. Cela suggère que `emails_dir` est un objet de type `Path` (probablement de la bibliothèque `pathlib`).
2. **Lecture du fichier** : Le fichier est ouvert en mode lecture (`'r'` par défaut) et son contenu est lu avec `f.read()`.
3. **Affichage du contenu** : Le contenu est ensuite affiché avec `print(return_email)`.

### Exemple complet avec la gestion de fichiers en Python :

```python
from pathlib import Path

# Chemin du répertoire contenant les emails
emails_dir = Path("/chemin/vers/emails")

# Ouverture et lecture du fichier "01_return.txt"
with open(emails_dir / "01_return.txt") as f:
    return_email = f.read()

# Affichage du contenu du fichier
print(return_email)
```

### Explication supplémentaire :
- **`Path` de `pathlib`** : Utilisé pour manipuler les chemins de fichiers de manière portable.
- **`with open(...)`** : Le bloc `with` garantit que le fichier est automatiquement fermé après avoir été lu, même en cas d'erreur.
- **`f.read()`** : Lit tout le contenu du fichier sous forme de chaîne.

Cela va afficher le texte contenu dans le fichier `"01_return.txt"`.

In [ ]:
query = prompt.format(email=return_email)
messages = [
    (
        "system",
        "You are a helpful assistant that processes orders from a bookstore.",
    ),
    ("human", query),
]
result = llm.invoke(query).content
print(result)

La question Python ici porte sur l'invocation d'un modèle de langage (`llm`) pour traiter un message formaté à partir d'un email contenu dans la variable `return_email`. Le processus consiste à formater le message en utilisant une structure de `prompt`, puis à envoyer ce message au modèle pour obtenir une réponse.

Voici une explication détaillée du processus, ainsi qu'un exemple de code montrant comment cela pourrait fonctionner :

### Explication :
1. **Formatage de la requête** : `query = prompt.format(email=return_email)` utilise la méthode `format` pour insérer le contenu de l'email (`return_email`) dans un `prompt` prédéfini, où le champ `{email}` est remplacé par l'email réel.
2. **Messages** : Une liste de messages est créée. Le premier message, `"system"`, donne des instructions au modèle, tandis que le second, `"human"`, contient la requête formatée (`query`).
3. **Invocation du modèle** : `llm.invoke(query)` est une invocation du modèle de langage (`llm`), qui renvoie un objet contenant le résultat dans l'attribut `content`.
4. **Affichage du résultat** : Le contenu de la réponse (`result.content`) est ensuite affiché avec `print()`.

### Exemple de code Python :

```python
# Supposons que 'llm' soit un modèle de langage déjà initialisé
class LLM:
    def invoke(self, query):
        # Simule la réponse du modèle
        return self.Response(f"Le résultat pour l'email est : {query}")
    
    class Response:
        def __init__(self, content):
            self.content = content

# Initialisation du modèle de langage
llm = LLM()

# Exemple de contenu du fichier email
return_email = """
Cher libraire,
Je voudrais retourner le livre "Python Avancé". Puis-je avoir des instructions sur le processus de retour ?
Merci.
"""

# Formatage du prompt
prompt = """
Given the email provided, please read it and analyse the contents.
If a name of a book is mentioned, return it.
If no name is mentioned, return empty string.
The email will be given between <email></email> XML tags.

<email>
{email}
</email>

Return the name of the book between <book></book> XML tags.
"""

# Formatage de la requête
query = prompt.format(email=return_email)

# Création des messages
messages = [
    ("system", "You are a helpful assistant that processes orders from a bookstore."),
    ("human", query),
]

# Invocation du modèle et récupération du résultat
result = llm.invoke(query).content

# Affichage du résultat
print(result)
```

### Explication :
- Le modèle `LLM` est simulé pour répondre à la requête formatée. Dans un vrai scénario, `llm.invoke()` ferait appel à une API de modèle de langage pour analyser et traiter la requête.
- Le contenu de l'email est inséré dans le `prompt` à l'endroit de `{email}`, et le modèle analyse cette requête.
- Le résultat du modèle est récupéré et affiché avec `print(result)`.

Ce code montre comment la chaîne de traitement fonctionne pour un modèle de langage interactif dans un contexte de gestion des commandes d'une librairie.

Using tags also allows us to extract multiple pieces of information at the same time and makes extraction much easier.
In the following prompt we will extract not just the book name, but any questions, requests and customer name.

In [ ]:
prompt = """

Human: Given email provided , please read it and analyse the contents.

Please extract the following information from the email:
- Any questions the customer is asking, return it inside <questions></questions> XML tags.
- The customer full name, return it inside <name></name> XML tags.
- Any book names the customer mentions, return it inside <books></books> XML tags.

If a particular bit of information is not present, return an empty string.
Make sure that each question can be understoon by itself, incorporate context if requred.
Each returned question should be concise, remove extra information if possible.
The email will be given between <email></email> XML tags.

<email>
{email}
</email>

Return each question inside <question></question> XML tags.
Return the name of each book inside <book></book> XML tags.

Assistant:"""

In [ ]:
query = prompt.format(email=book_question_email)
messages = [
    (
        "system",
        "You are a helpful assistant that processes orders from a bookstore.",
    ),
    ("human", query),
]
result = llm.invoke(query).content
print(result)

La question Python ici consiste à formater une requête en insérant le contenu de l'email dans un modèle de langage, à envoyer cette requête via une invocation du modèle (`llm.invoke(query)`), puis à afficher le résultat. Le processus commence par la construction d'un message formaté à l'aide de `prompt.format()` et continue avec l'interaction avec un assistant (système) pour traiter une commande.

Voici un exemple de code illustrant cela, avec un modèle de langage fictif pour simuler la réponse.

### Exemple complet :

```python
# Supposons que 'llm' soit un modèle de langage déjà initialisé
class LLM:
    def invoke(self, query):
        # Simulation de la réponse du modèle à partir de la requête
        return self.Response(f"Voici le livre que vous cherchez : 'Python pour les débutants'. Requête : {query}")
    
    class Response:
        def __init__(self, content):
            self.content = content

# Initialisation du modèle fictif
llm = LLM()

# Exemple de contenu de l'email contenant une question sur un livre
book_question_email = """
Bonjour, je voudrais avoir des informations sur le livre 'Python pour les débutants'.
Pouvez-vous me fournir plus de détails ?
"""

# Prompt avec un espace pour insérer l'email
prompt = """
Given the email provided, please read it and analyse the contents.
If a name of a book is mentioned, return it.
If no name is mentioned, return empty string.
The email will be given between <email></email> XML tags.

<email>
{email}
</email>

Return the name of the book between <book></book> XML tags.
"""

# Formatage de la requête
query = prompt.format(email=book_question_email)

# Création des messages à envoyer au modèle
messages = [
    ("system", "You are a helpful assistant that processes orders from a bookstore."),
    ("human", query),
]

# Invocation du modèle et récupération du résultat
result = llm.invoke(query).content

# Affichage du résultat
print(result)
```

### Explication :
1. **`prompt.format(email=book_question_email)`** : Formate le `prompt` en insérant le contenu de l'email (`book_question_email`) dans la place réservée `{email}`.
2. **`messages`** : Une liste contenant les rôles du système et de l'humain, où le message de l'humain est la requête formatée (`query`).
3. **`llm.invoke(query)`** : Le modèle de langage reçoit la requête formatée et renvoie un résultat sous forme d'une chaîne `result.content`.
4. **Affichage** : Le résultat est affiché avec `print(result)`.

Cela simule un assistant qui traite une commande pour un livre mentionné dans un email.

In [ ]:
extract_by_tag(result, "question", extract_all=True)

In [ ]:
extract_by_tag(result, "name")

In [ ]:
extract_by_tag(result, "book", extract_all=True)

## Conclusion

Entity extraction is a powerful technique using which you can extract arbitrary data using plain text descriptions.

This is particularly useful when you need to extract specific data which doesn't have clear structure. In such cases regex and other traditional extraction techniques can be very difficult to implement.

### Take aways
- Adapt this notebook to experiment with different models available through Amazon Bedrock such as Amazon Titan and AI21 Labs Jurassic models.
- Change the prompts to your specific usecase and evaluate the output of different models.
- Apply different prompt engineering principles to get better outputs. Refer to the prompt guide for your chosen model for recommendations, e.g. [here is the prompt guide for Claude](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design).